# Using Neon as a vector database for OpenAI embeddings

This notebook guides you step by step on using Neon Serverless Postgres as a vector database for OpenAI embeddings.

This notebook presents an end-to-end process of:
1. Using precomputed embeddings created by OpenAI API.
2. Storing the embeddings in a Neon database.
3. Converting raw text query to an embedding with OpenAI API.
4. Using Neon with the `pgvector` extension to perform the nearest neighbour search in the created collection.

### What is Neon Serverless Postgres

[Neon Serverless Postgres](https://www.alibabacloud.com/help/en/polardb/latest/what-is-polardb-2) is open-source serverless Postgres built for the cloud. Neon separates compute and storage to offer modern developer features such as autoscaling, database branching, scale-to-zero, and more. Neon supports vector search through the [pg_embedding](https://neon.tech/docs/extensions/pg_embedding) and [pgvector](https://neon.tech/docs/extensions/pgvector) open-source PostgreSQL extensions, both of which allow you to enable Postgres as a vector database.;







### Deployment options

- Neon is a cloud-hosted Postgres database service. You can create a Free Tier account and set up a project with a default database in a few simple steps. For instructions, see [Sign up](https://neon.tech/docs/get-started-with-neon/signing-up) and [Create your first project](https://neon.tech/docs/get-started-with-neon/setting-up-a-project).

## Prerequisites

For the purposes of this exercise we need to prepare the following:

1. A Neon Postgres database. Refer to the links provided above to set up an account and a Neon project.
2. The connection string for your Neon database. For infomration about obtaining a connection string, see [Connect from any application](https://neon.tech/docs/connect/connect-from-any-app).
2. You have installed the `pgvector` extension in Neon by running `CREATE EXTENSION vector;`. For instructions, see [Enable the pgvector exstenion](https://neon.tech/docs/extensions/pgvector#enable-the-pgvector-extension). 
2. The 'psycopg2' library to interact with Neon. Any other PostgreSQL client library is also supported.
3. An [OpenAI API key](https://platform.openai.com/account/api-keys).

### Setup requirements

This notebook requires the `openai` and `psycopg2` packages, but there are also some other additional libraries we will use. The following command installs them all:


In [3]:
! pip install openai psycopg2 pandas wget

'pip' is not recognized as an internal or external command,
operable program or batch file.


### Prepare your OpenAI API key

The OpenAI API key is used for vectorization of the documents and queries.

If you don't have an OpenAI API key, you can get one from https://platform.openai.com/account/api-keys.

Once you have your key, add it to your environment variables as `OPENAI_API_KEY`.

For additional information about setting your OpenAI API as an environment variable, please refer to [Best Practices for API Key Safety](https://help.openai.com/en/articles/5112595-best-practices-for-api-key-safety).

### Test your OpenAPI key

In [5]:
# Test that your OpenAI API key is set correctly as an environment variable
# Note: If you run this notebook locally, you will need to reload your terminal and the notebook for the environment variables to be available.

if os.getenv("OPENAI_API_KEY") is not None:
    print("OPENAI_API_KEY is ready")
else:
    print("OPENAI_API_KEY environment variable not found")

OPENAI_API_KEY environment variable not found


## Connect to your Neon database

Add the connection information defined below to your environment variables. Alternatively, you can just modify the "psycopg2.connect" parameters below.

Connecting to your Neon database server is easy with the official Python library:

In [6]:
import os
import psycopg2

# Note. alternatively you can set a temporary env variable like this:
# os.environ["PGHOST"] = "ep-snowy-unit-550577.us-east-2.aws.neon.tech"
# os.environ["PGPORT"] "5432"),
# os.environ["PGDATABASE"] "neondb"),
# os.environ["PGUSER"] "user"),
# os.environ["PGPASSWORD"] "password"),

connection = psycopg2.connect(
    host=os.environ.get("PGHOST", "ep-square-grass-34421519.ap-southeast-1.aws.neon.tech"),
    port=os.environ.get("PGPORT", "5432"),
    database=os.environ.get("PGDATABASE", "neondb"),
    user=os.environ.get("PGUSER", "daniel"),
    password=os.environ.get("PGPASSWORD", "Ag8MSyvUPIx9")
)

# Create a new cursor object
cursor = connection.cursor()

ModuleNotFoundError: No module named 'psycopg2'

Test the connection by running any available method:

In [5]:
# Execute a simple query to test the connection
cursor.execute("SELECT 1;")
result = cursor.fetchone()

# Check the query result
if result == (1,):
    print("Connection successful!")
else:
    print("Connection failed.")

Connection successful!


Import the pre-computed embeddings archive file:

In [7]:
import wget

embeddings_url = "https://cdn.openai.com/API/examples/data/vector_database_wikipedia_articles_embedded.zip"

# The file is ~700 MB so this will take some time
wget.download(embeddings_url)

'vector_database_wikipedia_articles_embedded.zip'

Extract the downloaded archive file:

In [8]:
import zipfile
import os
import re
import tempfile

current_directory = os.getcwd()
zip_file_path = os.path.join(current_directory, "vector_database_wikipedia_articles_embedded.zip")
output_directory = os.path.join(current_directory, "../../data")

with zipfile.ZipFile(zip_file_path, "r") as zip_ref:
    zip_ref.extractall(output_directory)


# check the csv file exist
file_name = "vector_database_wikipedia_articles_embedded.csv"
data_directory = os.path.join(current_directory, "../../data")
file_path = os.path.join(data_directory, file_name)


if os.path.exists(file_path):
    print(f"The file {file_name} exists in the data directory.")
else:
    print(f"The file {file_name} does not exist in the data directory.")

The file vector_database_wikipedia_articles_embedded.csv exists in the data directory.


## Index data

The vector table will be called **articles** and each object will have both **title** and **content** vectors. 

We will start by creating a table with an index defined on both the **title** and **content** vector columns. We will then we then populate the table with our precomputed embeddings.

In [6]:
create_table_sql = '''
CREATE TABLE IF NOT EXISTS public.articles (
    id INTEGER NOT NULL,
    url TEXT,
    title TEXT,
    content TEXT,
    title_vector vector(1536),
    content_vector vector(1536),
    vector_id INTEGER
);

ALTER TABLE public.articles ADD PRIMARY KEY (id);
'''

# SQL statement for creating indexes
create_indexes_sql = '''
CREATE INDEX ON public.articles USING ivfflat (content_vector) WITH (lists = 1000);

CREATE INDEX ON public.articles USING ivfflat (title_vector) WITH (lists = 1000);
'''

# Execute the SQL statements
cursor.execute(create_table_sql)
cursor.execute(create_indexes_sql)

# Commit the changes
connection.commit()

## Load data

In this section we are going to load the data prepared previous to this session, so you don't have to recompute the embeddings of Wikipedia articles with your own credits.

In [ ]:
import io

# Path to your local CSV file
csv_file_path = '../../data/vector_database_wikipedia_articles_embedded.csv'

# Define a generator function to process the file line by line
def process_file(file_path):
    with open(file_path, 'r') as file:
        for line in file:
            yield line

# Create a StringIO object to store the modified lines
modified_lines = io.StringIO(''.join(list(process_file(csv_file_path))))

# Create the COPY command for the copy_expert method
copy_command = '''
COPY public.articles (id, url, title, content, title_vector, content_vector, vector_id)
FROM STDIN WITH (FORMAT CSV, HEADER true, DELIMITER ',');
'''

# Execute the COPY command using the copy_expert method
cursor.copy_expert(copy_command, modified_lines)

# Commit the changes
connection.commit()

In [9]:
# Check the collection size to make sure all the points have been stored
count_sql = """select count(*) from public.articles;"""
cursor.execute(count_sql)
result = cursor.fetchone()
print(f"Count:{result[0]}")

Count:25000


## Search data

Once the data is stored in your Neon database, you can query the collection for the closest vectors. We  provide an additional parameter `vector_name` to switch from title to content based search. Since the precomputed embeddings were created with `text-embedding-ada-002` OpenAI model, we also have to use it during search.

In [10]:
def query_neon(query, collection_name, vector_name="title_vector", top_k=20):

    # Creates embedding vector from user query
    embedded_query = openai.Embedding.create(
        input=query,
        model="text-embedding-ada-002",
    )["data"][0]["embedding"]

    # Convert the embedded_query to PostgreSQL compatible format
    embedded_query_pg = "[" + ",".join(map(str, embedded_query)) + "]"

    # Create SQL query
    query_sql = f"""
    SELECT id, url, title, l2_distance({vector_name},'{embedded_query_pg}'::VECTOR(1536)) AS similarity
    FROM {collection_name}
    ORDER BY {vector_name} <-> '{embedded_query_pg}'::VECTOR(1536)
    LIMIT {top_k};
    """
    # Execute the query
    cursor.execute(query_sql)
    results = cursor.fetchall()

    return results

In [11]:
import openai

query_results = query_neon("modern art in Europe", "Articles")
for i, result in enumerate(query_results):
    print(f"{i + 1}. {result[2]} (Score: {round(1 - result[3], 3)})")

1. Museum of Modern Art (Score: 0.5)
2. Western Europe (Score: 0.485)
3. Renaissance art (Score: 0.479)
4. Pop art (Score: 0.472)
5. Northern Europe (Score: 0.461)
6. Hellenistic art (Score: 0.457)
7. Modernist literature (Score: 0.447)
8. Art film (Score: 0.44)
9. Central Europe (Score: 0.439)
10. European (Score: 0.437)
11. Art (Score: 0.437)
12. Byzantine art (Score: 0.436)
13. Postmodernism (Score: 0.434)
14. Eastern Europe (Score: 0.433)
15. Europe (Score: 0.433)
16. Cubism (Score: 0.432)
17. Impressionism (Score: 0.432)
18. Bauhaus (Score: 0.431)
19. Surrealism (Score: 0.429)
20. Expressionism (Score: 0.429)


In [12]:
# This time we'll query using content vector
query_results = query_neon("Famous battles in Scottish history", "Articles", "content_vector")
for i, result in enumerate(query_results):
    print(f"{i + 1}. {result[2]} (Score: {round(1 - result[3], 3)})")

1. Battle of Bannockburn (Score: 0.489)
2. Wars of Scottish Independence (Score: 0.474)
3. 1651 (Score: 0.457)
4. First War of Scottish Independence (Score: 0.452)
5. Robert I of Scotland (Score: 0.445)
6. 841 (Score: 0.441)
7. 1716 (Score: 0.441)
8. 1314 (Score: 0.429)
9. 1263 (Score: 0.428)
10. William Wallace (Score: 0.426)
11. Stirling (Score: 0.419)
12. 1306 (Score: 0.419)
13. 1746 (Score: 0.418)
14. 1040s (Score: 0.414)
15. 1106 (Score: 0.412)
16. 1304 (Score: 0.411)
17. David II of Scotland (Score: 0.408)
18. Braveheart (Score: 0.407)
19. 1124 (Score: 0.406)
20. July 27 (Score: 0.405)
